## 数据预处理

### 取pull数据

In [1]:
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np

# ACCESS_ID SECRET_ACCESS_KEYvODPS_PROJECT ODPS_ENDPOINT github提示有敏感信息无法上传，数据集来源为阿里云
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False

In [45]:
TEMP_TABLE_NAME0 = 'hx_repo_actor_birank_pull_id_and_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        repo_name    STRING,
        actor_id    BIGINT,
        actor_login    STRING,
        created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022051901373548gfzrj9jb>

In [46]:
# 把时间改掉
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME}
        SELECT
            repo_id,
            repo_name,
            actor_id,
            actor_login,
            created_at
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'opened') AND actor_login NOT LIKE '%bot%'
        AND pt>='20160101' AND pt<'20220101'
        ORDER BY repo_id, actor_id'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME0)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [47]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME0))
data.head(10)

,repo_id,repo_name,actor_id,actor_login,created_at
0,1,mojombo/grit,24193253,ankitts12,2020-10-01 11:55:32
1,27,rubinius/rubinius,85,brixen,2016-02-06 02:05:54
2,27,rubinius/rubinius,85,brixen,2016-02-05 02:38:10
3,27,rubinius/rubinius,85,brixen,2016-06-19 04:39:25
4,27,rubinius/rubinius,85,brixen,2016-06-05 02:04:47
5,27,rubinius/rubinius,85,brixen,2016-05-01 10:43:10
6,27,rubinius/rubinius,85,brixen,2016-07-02 00:17:11
7,27,rubinius/rubinius,85,brixen,2016-05-21 07:27:56
8,27,rubinius/rubinius,85,brixen,2016-05-06 07:47:28
9,27,rubinius/rubinius,85,brixen,2016-05-01 14:19:25


In [5]:
select_all = '''SELECT * FROM {TEMP_TABLE_NAME}'''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME0)
table_all = o.execute_sql(select_all, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})

In [5]:
TEMP_TABLE_NAME1 = 'hx_repo_actor_birank_pull'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        actor_id    BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519005153139gfelz4s92>

In [6]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT DISTINCT
            repo_id,
            actor_id
        FROM {TEMP_TABLE_NAME1}
        ORDER BY repo_id, actor_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME0,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME1
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [7]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME1))
data.head(10)

,repo_id,actor_id
0,1,24193253
1,27,85
2,27,10416
3,27,16881
4,27,18191
5,27,25555
6,27,39406
7,27,43569
8,27,48936
9,27,79817


### 重新编号

#### repo_id重新编号

In [8]:
TEMP_TABLE_NAME2 = 'hx_repo_actor_birank_pull_new_repoid_pre'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022051900531293g57mz4s92>

In [9]:
insert_new_repoid_pre_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT DISTINCT repo_id
        FROM {TEMP_TABLE_NAME1}
        ORDER BY repo_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME2
)
table = o.execute_sql(insert_new_repoid_pre_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [10]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME2))
data.head(15)

,repo_id
0,1
1,27
2,28
3,31
4,35
5,43
6,68
7,98
8,119
9,129


In [11]:
TEMP_TABLE_NAME3 = 'hx_repo_actor_birank_pull_new_repoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        new_repo_id    BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519005335524gzu8c1z9>

In [12]:
insert_new_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT repo_id, row_number() over() - 1 as new_repo_id
        FROM {TEMP_TABLE_NAME1}
        ORDER BY repo_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME2,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME3
)
table = o.execute_sql(insert_new_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [13]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME3))
data.head(15)

,repo_id,new_repo_id
0,1,0
1,27,1
2,28,2
3,31,3
4,35,4
5,43,5
6,68,6
7,98,7
8,119,8
9,129,9


In [14]:
# 查看最大new_repo_id(用于保证与WATCH中的new_repo_id一致)
select_max_new_repo_id = '''SELECT MAX(new_repo_id) FROM {TEMP_TABLE_NAME}'''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME3)
table_max_new_repo_id = o.execute_sql(select_max_new_repo_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_repo_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   14800777
}


#### actor_id重新编号

In [15]:
TEMP_TABLE_NAME4 = 'hx_repo_actor_birank_pull_new_actorid_pre'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519005435191gqb2pnim>

In [16]:
insert_new_actorid_pre_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT DISTINCT actor_id
        FROM {TEMP_TABLE_NAME1}
        ORDER BY actor_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME4
)
table = o.execute_sql(insert_new_actorid_pre_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [17]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME4))
data.head(15)

,actor_id
0,1
1,2
2,4
3,6
4,7
5,17
6,18
7,19
8,20
9,21


In [18]:
TEMP_TABLE_NAME5 = 'hx_repo_actor_birank_pull_new_actorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        new_actor_id    BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022051900545346gpg2pnim>

In [19]:
insert_new_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT actor_id, row_number() over() - 1 as new_actor_id
        FROM {TEMP_TABLE_NAME1}
        ORDER BY actor_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME4,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME5
)
table = o.execute_sql(insert_new_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [20]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME5))
data.head(15)

,actor_id,new_actor_id
0,1,0
1,2,1
2,4,2
3,6,3
4,7,4
5,17,5
6,18,6
7,19,7
8,20,8
9,21,9


In [21]:
# 查看最大new_actor_id(用于保证与follower中的new_actor_id一致)
select_max_new_actor_id = '''SELECT MAX(new_actor_id) FROM {TEMP_TABLE_NAME}'''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME5)
table_max_new_actor_id = o.execute_sql(select_max_new_actor_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_actor_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   7282527
}


### weight矩阵

In [22]:
TEMP_TABLE_NAME6 = 'hx_repo_actor_birank_pull_weight'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_repo_id    BIGINT,
        new_actor_id     BIGINT,
        weight    BIGINT DEFAULT 1
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME6
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519005513481gikrzh8a2>

In [23]:
weight_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME4}(new_repo_id, new_actor_id)
        SELECT new_repo_id, new_actor_id
        FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}, {TEMP_TABLE_NAME3}
        WHERE {TEMP_TABLE_NAME1}.repo_id = {TEMP_TABLE_NAME2}.repo_id AND {TEMP_TABLE_NAME1}.actor_id = {TEMP_TABLE_NAME3}.actor_id
        ORDER BY new_repo_id, new_actor_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME3,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME5,
    TEMP_TABLE_NAME4=TEMP_TABLE_NAME6
)
table = o.execute_sql(weight_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [24]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME6))
data.head(15)

,new_repo_id,new_actor_id,weight
0,0,2737157,1
1,1,44,1
2,1,5950,1
3,1,8882,1
4,1,9452,1
5,1,12528,1
6,1,17669,1
7,1,19274,1
8,1,21396,1
9,1,32244,1


In [25]:
pd_weight = data.to_pandas()
pd_weight.head()

,new_repo_id,new_actor_id,weight
0,0,2737157,1
1,1,44,1
2,1,5950,1
3,1,8882,1
4,1,9452,1


### new_id与name对应

#### new_repo_id与repo_name对应

In [26]:
TEMP_TABLE_NAME8_1 = 'hx_repo_actor_birank_pull_repo_id_and_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        repo_name    STRING,
        created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME8_1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519011629433g7icd9jb>

In [28]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT DISTINCT
            repo_id,
            repo_name,
            created_at
        FROM {TEMP_TABLE_NAME1}
        ORDER BY repo_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME0,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME8_1)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [29]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME8_1))
data.head(10)

,repo_id,repo_name,created_at
0,1,mojombo/grit,2020-10-01 11:55:32
1,27,rubinius/rubinius,2016-01-15 04:49:23
2,27,rubinius/rubinius,2016-01-15 20:54:02
3,27,rubinius/rubinius,2016-01-16 04:30:24
4,27,rubinius/rubinius,2016-01-16 07:14:21
5,27,rubinius/rubinius,2016-01-16 19:26:16
6,27,rubinius/rubinius,2016-01-16 22:58:34
7,27,rubinius/rubinius,2016-01-17 05:38:53
8,27,rubinius/rubinius,2016-01-21 03:07:33
9,27,rubinius/rubinius,2016-01-21 08:39:55


In [30]:
TEMP_TABLE_NAME8_2 = 'hx_repo_actor_birank_pull_repo_max_created_at'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        max_created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME8_2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519011951463gxuq51z9>

In [31]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT
            repo_id,
            MAX(created_at)
        FROM {TEMP_TABLE_NAME1}
        GROUP BY repo_id
        ORDER BY repo_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME8_1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME8_2
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [32]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME8_2))
data.head(10)

,repo_id,max_created_at
0,1,2020-10-01 11:55:32
1,27,2020-11-12 05:53:20
2,28,2017-11-11 06:42:24
3,31,2020-10-17 14:33:59
4,35,2016-01-01 12:57:28
5,43,2017-03-28 08:48:29
6,68,2021-09-01 11:35:51
7,98,2016-04-23 08:29:05
8,119,2020-10-01 18:22:05
9,129,2021-10-02 00:56:07


In [33]:
TEMP_TABLE_NAME8_3 = 'hx_repo_actor_birank_pull_repo_newest_repo_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        repo_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME8_3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519012228837gz88cta2>

In [34]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME3}
        SELECT
            {TEMP_TABLE_NAME2}.repo_id,
            repo_name
        FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
        WHERE {TEMP_TABLE_NAME1}.repo_id = {TEMP_TABLE_NAME2}.repo_id AND {TEMP_TABLE_NAME1}.max_created_at = {TEMP_TABLE_NAME2}.created_at
        ORDER BY repo_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME8_2,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME8_1,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME8_3
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [35]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME8_3))
data.head(10)

,repo_id,repo_name
0,1,mojombo/grit
1,27,rubinius/rubinius
2,28,mojombo/god
3,31,wycats/jspec
4,35,defunkt/exception_logger
5,43,technoweenie/attachment_fu
6,68,macournoyer/thin
7,98,bmizerany/sinatra
8,119,jnewland/gsa-feeds
9,129,jnewland/xhtmlize


In [36]:
TEMP_TABLE_NAME8 = 'hx_repo_actor_birank_pull_repo_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_repo_id     BIGINT,
        repo_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME8
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519012409870gmepxh8a2>

In [37]:
repo_newid_name_sql = '''set odps.sql.validate.orderby.limit=false;
    INSERT INTO {TEMP_TABLE_NAME3}
    SELECT DISTINCT new_repo_id, repo_name
    FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
    WHERE {TEMP_TABLE_NAME1}.repo_id = {TEMP_TABLE_NAME2}.repo_id
    ORDER BY new_repo_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME3,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME8_3,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME8
)
table = o.execute_sql(repo_newid_name_sql, hints={"odps.sql.submit.mode" : "script"})

In [38]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME8))
data.head(10)

,new_repo_id,repo_name
0,0,mojombo/grit
1,1,rubinius/rubinius
2,2,mojombo/god
3,3,wycats/jspec
4,4,defunkt/exception_logger
5,5,technoweenie/attachment_fu
6,6,macournoyer/thin
7,7,bmizerany/sinatra
8,8,jnewland/gsa-feeds
9,9,jnewland/xhtmlize


In [39]:
select_1614471 = '''SELECT max(new_repo_id) FROM {TEMP_TABLE_NAME}'''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME8)
table_1614471 = o.execute_sql(select_1614471, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_1614471.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   14800777
}


In [40]:
pd_repo_newid_name = data.to_pandas()
pd_repo_newid_name.head()

,new_repo_id,repo_name
0,0,mojombo/grit
1,1,rubinius/rubinius
2,2,mojombo/god
3,3,wycats/jspec
4,4,defunkt/exception_logger


#### new_actor_id与actor_name对应

In [41]:
TEMP_TABLE_NAME9_1 = 'hx_repo_actor_birank_pull_actor_id_and_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        actor_name    STRING,
        created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME9_1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519013231269ghk2c4s92>

In [48]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT DISTINCT
            actor_id,
            actor_login,
            created_at
        FROM {TEMP_TABLE_NAME1}
        ORDER BY actor_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME0,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME9_1)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [49]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME9_1))
data.head(10)

,actor_id,actor_name,created_at
0,1,mojombo,2018-11-19 09:47:34
1,1,mojombo,2018-12-02 09:54:39
2,1,mojombo,2019-11-17 05:22:47
3,1,mojombo,2019-12-01 15:07:01
4,1,mojombo,2019-12-05 14:55:20
5,1,mojombo,2019-12-22 09:29:02
6,1,mojombo,2020-01-08 21:15:44
7,1,mojombo,2020-01-11 04:52:38
8,1,mojombo,2020-01-16 05:29:50
9,1,mojombo,2020-01-18 07:02:10


In [50]:
TEMP_TABLE_NAME9_2 = 'hx_repo_actor_birank_pull_actor_max_created_at'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        max_created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME9_2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519014519436gftsppa2>

In [51]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT
            actor_id,
            MAX(created_at)
        FROM {TEMP_TABLE_NAME1}
        GROUP BY actor_id
        ORDER BY actor_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME9_1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME9_2
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [52]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME9_2))
data.head(10)

,actor_id,max_created_at
0,1,2021-11-10 20:42:36
1,2,2016-02-17 06:49:25
2,4,2021-12-24 16:49:02
3,6,2017-05-02 04:40:07
4,7,2021-11-30 11:24:43
5,17,2021-12-29 17:18:32
6,18,2021-11-30 17:08:37
7,19,2020-03-05 09:05:54
8,20,2021-12-26 21:46:04
9,21,2021-07-04 07:05:05


In [53]:
TEMP_TABLE_NAME9_3 = 'hx_repo_actor_birank_pull_repo_newest_actor_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        actor_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME9_3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519014543905gxr8e4s92>

In [54]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME3}
        SELECT
            {TEMP_TABLE_NAME2}.actor_id,
            actor_name
        FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
        WHERE {TEMP_TABLE_NAME1}.actor_id = {TEMP_TABLE_NAME2}.actor_id AND {TEMP_TABLE_NAME1}.max_created_at = {TEMP_TABLE_NAME2}.created_at
        ORDER BY actor_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME9_2,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME9_1,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME9_3
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [55]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME9_3))
data.head(10)

,actor_id,actor_name
0,1,mojombo
1,2,defunkt
2,4,wycats
3,6,ivey
4,7,evanphx
5,17,vanpelt
6,18,wayneeseguin
7,19,brynary
8,20,kevinclark
9,21,technoweenie


In [56]:
TEMP_TABLE_NAME10 = 'hx_repo_actor_birank_pull_new_actorid_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_actor_id     BIGINT,
        actor_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME10
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519014616365goa9e4s92>

In [57]:
actor_newid_name_sql = '''set odps.sql.validate.orderby.limit=false;
    INSERT INTO {TEMP_TABLE_NAME3}
    SELECT DISTINCT new_actor_id, actor_name
    FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
    WHERE {TEMP_TABLE_NAME1}.actor_id = {TEMP_TABLE_NAME2}.actor_id
    ORDER BY new_actor_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME5,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME9_3,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME10
)
table = o.execute_sql(actor_newid_name_sql, hints={"odps.sql.submit.mode" : "script"})

In [58]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME10))
data.head(10)

,new_actor_id,actor_name
0,0,mojombo
1,1,defunkt
2,2,wycats
3,3,ivey
4,4,evanphx
5,5,vanpelt
6,6,wayneeseguin
7,7,brynary
8,8,kevinclark
9,9,technoweenie


In [59]:
pd_actor_newid_name = data.to_pandas()
pd_actor_newid_name.head()

,new_actor_id,actor_name
0,0,mojombo
1,1,defunkt
2,2,wycats
3,3,ivey
4,4,evanphx


pd_repo_newid_name存放了new_repo_id---repo_name的pd.df

pd_actor_newid_name存放了new_actor_id---actor_name的pd.df

## 算法

### birank

In [60]:
import numpy as np
import scipy
import scipy.sparse as spa
# 为了不破坏new_pd_data原始数据，这里用test_data作为new_pd_data的副本加入运算
test_data = pd_weight.copy()
row = test_data['new_actor_id'].values
col = test_data['new_repo_id'].values
data = test_data['weight'].values
# data=[1]*len(test_data['weight'])
# Weight矩阵是大小为Actors*Repos的稀疏矩阵,没有对应weight值的地方全为0
Weight = spa.coo_matrix((data,(row,col)))
print(Weight)

  (2737157, 0)	1
  (44, 1)	1
  (5950, 1)	1
  (8882, 1)	1
  (9452, 1)	1
  (12528, 1)	1
  (17669, 1)	1
  (19274, 1)	1
  (21396, 1)	1
  (32244, 1)	1
  (34269, 1)	1
  (46143, 1)	1
  (56233, 1)	1
  (60920, 1)	1
  (127642, 1)	1
  (130519, 1)	1
  (134343, 1)	1
  (180788, 1)	1
  (238599, 1)	1
  (263020, 1)	1
  (269050, 1)	1
  (279676, 1)	1
  (288341, 1)	1
  (328661, 1)	1
  (357827, 1)	1
  :	:
  (1389704, 14800753)	1
  (1389704, 14800754)	1
  (1389704, 14800755)	1
  (1389704, 14800756)	1
  (1389704, 14800757)	1
  (1389704, 14800758)	1
  (1389704, 14800759)	1
  (1389704, 14800760)	1
  (1389704, 14800761)	1
  (1389704, 14800762)	1
  (1389704, 14800763)	1
  (1389704, 14800764)	1
  (1389704, 14800765)	1
  (1389704, 14800766)	1
  (6960787, 14800767)	1
  (1389704, 14800768)	1
  (1389704, 14800769)	1
  (1389704, 14800770)	1
  (1389704, 14800771)	1
  (1389704, 14800772)	1
  (1389704, 14800773)	1
  (1911181, 14800774)	1
  (1389704, 14800775)	1
  (1389704, 14800776)	1
  (1389704, 14800777)	1


In [61]:
def birank(W,normalier='HITS',alpha=0.85,beta=0.85,max_iter=200,tol=1.0e-4,verbose=False):
    '''
    输入:
        W::scipy's sparse matrix:Actors*Repos二分图网络的邻接矩阵
        alpha, beta::float:超参数
        max_iter::int:最大迭代次数
        tol::float:检查收敛性的容错度
        verbose::boolean:运行时是否打印迭代信息
    输出:
        a, r::numpy.ndarray:Actor和Repos的birank的值
    '''
    # W为A*R的矩阵,WT为R*A
    W = W.astype('float', copy=False)
    WT=W.T
    # Da_bi代表对角线元素为actor度平方根的倒数的对角矩阵,Dr_bi代表对角线元素为repo度平方根的倒数的对角矩阵
    # Sr代表Dr_bi*WT*Da_bi,Sa代表Da_bi*W*Dr_bi
    
    Da = scipy.array(W.sum(axis=1)).flatten() 
    Dr = scipy.array(W.sum(axis=0)).flatten()

    Da_ = spa.diags(1/Da)
    Dr_ = spa.diags(1/Dr)

    # if normalier == 'HITS':
    #     Sr = WT
    #     Sa = W
    if normalier == 'CoHITS':
        Sr = WT.dot(Da_)
        Sa = W.dot(Dr_)
    # elif normalier == 'BGRM':
    #     Sr = Dr_.dot(WT).dot(Da_)
    #     Sa = Sr.T
    elif normalier == 'BiRank':
        Da_bi=spa.diags(1/scipy.sqrt(Da))
        Dr_bi=spa.diags(1/scipy.sqrt(Dr))
        Sr=Dr_bi.dot(WT).dot(Da_bi)
        Sa=Sr.T
    
    # 随机初始化a0,r0
    a0 = np.repeat(1/len(Da),len(Da))
    r0 = np.repeat(1/len(Dr),len(Dr))
    a_last=a0.copy()
    r_last=r0.copy()
    end=0
    for i in range(max_iter):
        r = alpha * (Sr.dot(a_last)) + (1-alpha) * r0
        a = beta * (Sa.dot(r_last)) + (1-beta) * a0
        err_r=np.absolute(r-r_last).sum()
        err_a=np.absolute(a-a_last).sum()
        if verbose:
            print(
                "Iteration : {}; actor error: {}; repo error: {}".format(i, err_a, err_r)
            )
        if err_r < tol and err_a < tol:
            break
        a_last=a
        r_last=r
        end=i
    print(end)
    return a,r

In [62]:
actor_value,repo_value = birank(Weight,'BiRank')

C:\Users\hp\AppData\Local\Temp/ipykernel_13056/721346134.py:18: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  Da = scipy.array(W.sum(axis=1)).flatten()
C:\Users\hp\AppData\Local\Temp/ipykernel_13056/721346134.py:19: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  Dr = scipy.array(W.sum(axis=0)).flatten()
C:\Users\hp\AppData\Local\Temp/ipykernel_13056/721346134.py:34: DeprecationWarning: scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead
  Da_bi=spa.diags(1/scipy.sqrt(Da))
C:\Users\hp\AppData\Local\Temp/ipykernel_13056/721346134.py:35: DeprecationWarning: scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead
  Dr_bi=spa.diags(1/scipy.sqrt(Dr))


51


In [63]:
repo_value_list = repo_value.tolist()
repo = {'repo_value': repo_value_list}
repo_info = pd.DataFrame(repo)
repo_info.sort_values(by="repo_value" , inplace=True, ascending=False) 
repo_info['new_repo_id'] = repo_info.index
repo_info = repo_info.reset_index(drop=True)
repo_info['rank'] = repo_info.index+1
repo_info.head(10)


,repo_value,new_repo_id,rank
0,0.000008,1614471,1
1,0.000008,170062,2
2,0.000007,8262113,3
3,0.000005,17203,4
4,0.000005,3854851,5
5,0.000005,6432194,6
6,0.000005,2538677,7
7,0.000005,267844,8
8,0.000004,71898,9
9,0.000004,1136011,10


In [64]:
repo_result = pd.merge(repo_info, pd_repo_newid_name, on='new_repo_id')
order = ['new_repo_id', 'repo_name', 'repo_value','rank']
repo_result = repo_result[order]
repo_result.head(20)

,new_repo_id,repo_name,repo_value,rank
0,1614471,firstcontributions/first-contributions,0.000008,1
1,170062,jlord/patchwork,0.000008,2
2,8262113,google/it-cert-automation-practice,0.000007,3
3,17203,octocat/Spoon-Knife,0.000005,4
4,3854851,zero-to-mastery/start-here-guidelines,0.000005,5
5,6432194,freddier/hyperblog,0.000005,6
6,2538677,udacity/course-collaboration-travel-plans,0.000005,7
7,267844,JetBrains/swot,0.000005,8
8,71898,DefinitelyTyped/DefinitelyTyped,0.000004,9
9,1136011,githubschool/open-enrollment-classes-introduct...,0.000004,10


In [65]:
actor_value_list = actor_value.tolist()
actor = {'actor_value': actor_value_list}
actor_info = pd.DataFrame(actor)
actor_info.sort_values(by="actor_value" , inplace=True, ascending=False) 
actor_info['new_actor_id'] = actor_info.index
actor_info = actor_info.reset_index(drop=True)
actor_info['rank'] = actor_info.index+1
actor_result = pd.merge(actor_info, pd_actor_newid_name, on='new_actor_id')
order = ['new_actor_id', 'actor_name', 'actor_value','rank']
actor_result = actor_result[order]
actor_result.head(20)

,new_actor_id,actor_name,actor_value,rank
0,1389704,direwolf-github,0.000040,1
1,1065416,xmo-odoo,0.000006,2
2,5631549,codeserver-test1,0.000005,3
3,2730410,codacy-badger,0.000005,4
4,1172471,gitter-badger,0.000004,5
5,2263149,QualitySoftwareDeveloper,0.000003,6
6,4823957,git2e,0.000003,7
7,2121899,idsb3t1,0.000003,8
8,5148049,codeserver-service-qa,0.000003,9
9,6575172,turbocanary-admin,0.000003,10


In [ ]:
birank_repo_top100 = pd.DataFrame(repo_result, columns=['new_repo_id','repo_name'])[:100]
birank_repo_top100.to_csv('./birank_repo_idname_top100.csv')
birank_actor_top100 = pd.DataFrame(actor_result, columns=['new_actor_id','actor_name'])[:100]
birank_actor_top100.to_csv('./birank_actor_idname_top100.csv')

### pagerank

对actor_id重新编号，使之变为原id+repo的数量

In [293]:
# 求repo的数量
repo_number=len(pd_weight['new_repo_id'].unique())
print(repo_number)
print(pd_weight['new_repo_id'].min())
print(pd_weight['new_repo_id'].max())
print(pd_weight['new_actor_id'].max())
pd_weight['new_actor_id'] = pd_weight['new_actor_id'].map(lambda x: x+repo_number)
print(pd_weight['new_actor_id'].min())
print(pd_weight['new_actor_id'].max())

14800778
0
14800777
7282527
14800778
22083305


In [294]:
# 计算节点数量
# 要获得邻接矩阵，还必须把对角线上的元素值设为1
adj = pd_weight.copy()
adj.columns = ['adj_row', 'adj_col', 'weight']
adj2 = pd_weight.copy()
adj2.columns = ['adj_col', 'adj_row', 'weight']
re_order = ['adj_row', 'adj_col', 'weight']
adj2 = adj2[re_order]
# print(adj)
# print(adj2)
node_number = adj['adj_col'].max()+1
print(node_number)
node_list = [i for i in range(0,node_number)]
diag_data = {'adj_row':node_list, 'adj_col':node_list, 'weight':[1]*node_number}
diag_data_df = pd.DataFrame(diag_data)
# diag_data_df.head(10)
concat_adj = pd.concat([adj, adj2, diag_data_df])
print(concat_adj)

22083306
           adj_row   adj_col  weight
0                0  17537935       1
1                1  14800822       1
2                1  14806728       1
3                1  14809660       1
4                1  14810230       1
...            ...       ...     ...
22083301  22083301  22083301       1
22083302  22083302  22083302       1
22083303  22083303  22083303       1
22083304  22083304  22083304       1
22083305  22083305  22083305       1

[77179940 rows x 3 columns]


In [295]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy
import scipy.sparse as spa
# 构建pagerank的稀疏矩阵
row = concat_adj['adj_row'].values
col = concat_adj['adj_col'].values
data = concat_adj['weight'].values
# data=[1]*len(test_data['weight'])
# Weight矩阵是大小为Actors*Repos的稀疏矩阵,没有对应weight值的地方全为0
Adj = spa.coo_matrix((data,(row,col)))
print(Adj)

  (0, 17537935)	1
  (1, 14800822)	1
  (1, 14806728)	1
  (1, 14809660)	1
  (1, 14810230)	1
  (1, 14813306)	1
  (1, 14818447)	1
  (1, 14820052)	1
  (1, 14822174)	1
  (1, 14833022)	1
  (1, 14835047)	1
  (1, 14846921)	1
  (1, 14857011)	1
  (1, 14861698)	1
  (1, 14928420)	1
  (1, 14931297)	1
  (1, 14935121)	1
  (1, 14981566)	1
  (1, 15039377)	1
  (1, 15063798)	1
  (1, 15069828)	1
  (1, 15080454)	1
  (1, 15089119)	1
  (1, 15129439)	1
  (1, 15158605)	1
  :	:
  (22083281, 22083281)	1
  (22083282, 22083282)	1
  (22083283, 22083283)	1
  (22083284, 22083284)	1
  (22083285, 22083285)	1
  (22083286, 22083286)	1
  (22083287, 22083287)	1
  (22083288, 22083288)	1
  (22083289, 22083289)	1
  (22083290, 22083290)	1
  (22083291, 22083291)	1
  (22083292, 22083292)	1
  (22083293, 22083293)	1
  (22083294, 22083294)	1
  (22083295, 22083295)	1
  (22083296, 22083296)	1
  (22083297, 22083297)	1
  (22083298, 22083298)	1
  (22083299, 22083299)	1
  (22083300, 22083300)	1
  (22083301, 22083301)	1
  (22083302, 220833

In [296]:
def pagerank(adj, d=0.85, max_iter=200, tol=1.0e-4, verbose=False):
    """
    Input:
        adj::scipy.sparsematrix:图的邻接矩阵
        d::float:参数
        max_iter::int:最大迭代的次数
        tol::float:检查收敛性的容错度
        verbose::boolean:运行时是否打印迭代信息
    Output:
        ::numpy.ndarray:The pageRank values
    """
    adj = adj.astype('float', copy=False)
    n_node = adj.shape[0]
    n_inverse = np.repeat(1.0 / n_node, n_node)
    S = np.array(adj.sum(axis=1)).flatten()
    S[S != 0] = 1.0 / S[S != 0]
    Q = spa.spdiags(S.T, 0, *adj.shape, format='csr')
    M = Q*adj

    x = np.repeat(1.0 / n_node, n_node)
    end=0
    for i in range(max_iter):
        xlast = x
        x = d * (x * M) + (1 - d) * n_inverse
        err = np.absolute(x - xlast).sum()
        end = i
        if verbose:
            print(i, err)
        if err < tol:
            break
    print(end)
    return x


In [297]:
node_value = pagerank(Adj)

21


In [298]:
print(type(node_value))
repo_value = node_value[0:repo_number]
print(len(repo_value))
print(type(repo_value))
actor_value = node_value[repo_number:]
print(len(actor_value))

<class 'numpy.ndarray'>
14800778
<class 'numpy.ndarray'>
7282528


In [299]:
repo_value_list = repo_value.tolist()
repo = {'repo_value': repo_value_list}
repo_pagerank_info = pd.DataFrame(repo)
repo_pagerank_info.sort_values(by="repo_value" , inplace=True, ascending=False) 
repo_pagerank_info['new_repo_id'] = repo_pagerank_info.index
repo_pagerank_info = repo_pagerank_info.reset_index(drop=True)
repo_pagerank_info['rank'] = repo_pagerank_info.index+1
repo_pagerank_info.head(10)


,repo_value,new_repo_id,rank
0,0.000362,1614471,1
1,0.000308,170062,2
2,0.000195,8262113,3
3,0.000139,17203,4
4,0.000117,3854851,5
5,0.000112,71898,6
6,0.000101,2538677,7
7,0.000100,6432194,8
8,0.000098,267844,9
9,0.000080,1136011,10


In [300]:
repo_pagerank_result = pd.merge(repo_pagerank_info, pd_repo_newid_name, on='new_repo_id')
order = ['new_repo_id', 'repo_name', 'repo_value','rank']
repo_pagerank_result = repo_pagerank_result[order]
repo_pagerank_result.head(10)

,new_repo_id,repo_name,repo_value,rank
0,1614471,firstcontributions/first-contributions,0.000362,1
1,170062,jlord/patchwork,0.000308,2
2,8262113,google/it-cert-automation-practice,0.000195,3
3,17203,octocat/Spoon-Knife,0.000139,4
4,3854851,zero-to-mastery/start-here-guidelines,0.000117,5
5,71898,DefinitelyTyped/DefinitelyTyped,0.000112,6
6,2538677,udacity/course-collaboration-travel-plans,0.000101,7
7,6432194,freddier/hyperblog,0.000100,8
8,267844,JetBrains/swot,0.000098,9
9,1136011,githubschool/open-enrollment-classes-introduct...,0.000080,10


In [301]:
actor_value_list = actor_value.tolist()
actor = {'actor_value': actor_value_list}
actor_pagerank_info = pd.DataFrame(actor)
actor_pagerank_info.sort_values(by="actor_value" , inplace=True, ascending=False) 
actor_pagerank_info['new_actor_id'] = actor_pagerank_info.index
actor_pagerank_info = actor_pagerank_info.reset_index(drop=True)
actor_pagerank_info['rank'] = actor_pagerank_info.index+1
actor_pagerank_result = pd.merge(actor_pagerank_info, pd_actor_newid_name, on='new_actor_id')
order = ['new_actor_id', 'actor_name', 'actor_value','rank']
actor_pagerank_result = actor_pagerank_result[order]
actor_pagerank_result.head(20)

,new_actor_id,actor_name,actor_value,rank
0,1389704,direwolf-github,0.022866,1
1,1065416,xmo-odoo,0.000496,2
2,5631549,codeserver-test1,0.000375,3
3,2730410,codacy-badger,0.000222,4
4,1172471,gitter-badger,0.000158,5
5,2263149,QualitySoftwareDeveloper,0.000157,6
6,4823957,git2e,0.000155,7
7,2121899,idsb3t1,0.000148,8
8,5148049,codeserver-service-qa,0.000146,9
9,6575172,turbocanary-admin,0.000117,10


## 统计信息

### watch

#### 查看ods_github_log表发现有名为WatchEvent的type，可以从中得到watch相关信息

In [61]:
TEMP_WATCH_TABLE00 = 'hx_repo_actor_birank_type'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        type    STRING
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE00
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220518150128260g9inue68>

In [62]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE}
        SELECT DISTINCT type
        FROM ods_github_log
        WHERE actor_login NOT LIKE '%bot%'
        AND pt>='20160101' AND pt<'20220101'
        '''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE00)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [63]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE00))
data.head()

,type
0,GollumEvent
1,PullRequestReviewCommentEvent
2,CommitCommentEvent
3,PublicEvent
4,IssueCommentEvent
5,ReleaseEvent
6,ForkEvent
7,PullRequestEvent
8,CreateEvent
9,MemberEvent


#### 统计各repo被watch的数目，建立 repo_id --- watch_cnt 对应关系表

In [66]:
TEMP_WATCH_TABLE1 = 'hx_repo_actor_birank_watch'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        repo_id     BIGINT,
        watch_cnt    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519015335233gsq2i592>

In [67]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT repo_id, count(*) AS watch_cnt
        FROM ods_github_log
        WHERE type="WatchEvent" AND repo_id IN (SELECT DISTINCT repo_id FROM {TEMP_WATCH_TABLE1})
        AND pt>='20160101' AND pt<'20220101'
        GROUP BY repo_id
        ORDER BY watch_cnt desc
        '''.format(
    TEMP_WATCH_TABLE1=TEMP_TABLE_NAME2,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE1)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [68]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE1))
data.head(10)

,repo_id,watch_cnt
0,177736533,367097
1,28457823,353140
2,60493101,222322
3,11730342,215106
4,85077558,196137
5,13491895,189269
6,54346799,181217
7,10270250,177594
8,45717250,171686
9,21737465,171290


#### 建立 repo_id --- new_repo_id 对应关系表

In [69]:
TEMP_WATCH_TABLE2_1 = 'hx_repo_actor_birank_pull_forwatch_only_repoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        repo_id     BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE2_1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519015713165gu620392>

In [70]:
get_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT DISTINCT repo_id
        FROM {TEMP_WATCH_TABLE1}
        ORDER BY repo_id
'''.format(
    TEMP_WATCH_TABLE1=TEMP_TABLE_NAME2,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE2_1
)
table = o.execute_sql(get_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [71]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE2_1))
data.head(15)

,repo_id
0,1
1,27
2,28
3,31
4,35
5,43
6,68
7,98
8,119
9,129


In [72]:
TEMP_WATCH_TABLE2 = 'hx_repo_actor_birank_pull_forwatch_new_repoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        repo_id     BIGINT,
        new_repo_id    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519015734138ghd4pnim>

In [73]:
insert_new_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT repo_id, row_number() over() - 1 as new_repo_id
        FROM {TEMP_WATCH_TABLE1}
        ORDER BY repo_id
'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE2_1,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE2
)
table = o.execute_sql(insert_new_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [74]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE2))
data.head(15)

,repo_id,new_repo_id
0,1,0
1,27,1
2,28,2
3,31,3
4,35,4
5,43,5
6,68,6
7,98,7
8,119,8
9,129,9


In [75]:
# 查看最大new_repo_id(用于保证与算法中的new_repo_id一致)
select_max_new_repo_id = '''SELECT MAX(new_repo_id) FROM {TEMP_WATCH_TABLE}'''.format(TEMP_WATCH_TABLE=TEMP_WATCH_TABLE2)
table_max_new_repo_id = o.execute_sql(select_max_new_repo_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_repo_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   14800777
}


#### 建立new_repo_id --- watch_cnt对应关系表

In [76]:
TEMP_WATCH_TABLE4 = 'hx_repo_actor_birank_newid_watch_cnt'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        new_repo_id    BIGINT,
        watch_cnt    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519015755111ghqp7yor2>

In [77]:
newid_watch_cnt_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE3}
        SELECT DISTINCT new_repo_id, watch_cnt
        FROM {TEMP_WATCH_TABLE1}, {TEMP_WATCH_TABLE2}
        WHERE {TEMP_WATCH_TABLE1}.repo_id = {TEMP_WATCH_TABLE2}.repo_id
        ORDER BY watch_cnt desc
'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE1,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE2,
    TEMP_WATCH_TABLE3=TEMP_WATCH_TABLE4
)
table = o.execute_sql(newid_watch_cnt_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [78]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE4))
data.head(15)

,new_repo_id,watch_cnt
0,6286308,367097
1,312191,353140
2,1193927,222322
3,128100,215106
4,2403462,196137
5,145820,189269
6,900570,181217
7,113507,177594
8,556665,171686
9,236321,171290


#### 建立new_repo_id --- watch_rank对应关系

In [79]:
TEMP_WATCH_TABLE5 = 'hx_repo_actor_birank_watch_rank'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        new_repo_id    BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519015939717ghpoh9jb>

In [80]:
insert_watch_rank_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT new_repo_id, row_number() over() AS rank
        FROM {TEMP_WATCH_TABLE1}
        ORDER BY rank
'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE4,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE5
)
table = o.execute_sql(insert_watch_rank_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [81]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE5))
data.head(10)

,new_repo_id,watch_rank
0,6286308,1
1,312191,2
2,1193927,3
3,128100,4
4,2403462,5
5,145820,6
6,900570,7
7,113507,8
8,556665,9
9,236321,10


In [82]:
pd_idwatch = data.to_pandas()
pd_idwatch.head()

,new_repo_id,watch_rank
0,6286308,1
1,312191,2
2,1193927,3
3,128100,4
4,2403462,5


In [83]:
watch_10 = pd_idwatch[:10]
watch_10

,new_repo_id,watch_rank
0,6286308,1
1,312191,2
2,1193927,3
3,128100,4
4,2403462,5
5,145820,6
6,900570,7
7,113507,8
8,556665,9
9,236321,10


In [84]:
watch_20 = pd_idwatch[:20]
watch_20

,new_repo_id,watch_rank
0,6286308,1
1,312191,2
2,1193927,3
3,128100,4
4,2403462,5
5,145820,6
6,900570,7
7,113507,8
8,556665,9
9,236321,10


In [85]:
watch_50 = pd_idwatch[:50]
watch_50

,new_repo_id,watch_rank
0,6286308,1
1,312191,2
2,1193927,3
3,128100,4
4,2403462,5
5,145820,6
6,900570,7
7,113507,8
8,556665,9
9,236321,10


#### 为方便展示，建立new_repo_id---repo_name---watch_rank对应关系表

In [86]:
TEMP_WATCH_TABLE6 = 'hx_repo_actor_birank_pull_watch_repo_withname_rank'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        new_repo_id     BIGINT,
        repo_name    STRING,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE6
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519020101562gf9s7yor2>

In [87]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE4}
        SELECT
            {TEMP_WATCH_TABLE1}.new_repo_id,
            repo_name,
            watch_rank
        FROM {TEMP_WATCH_TABLE1}, {TEMP_WATCH_TABLE2}, {TEMP_WATCH_TABLE3}
        WHERE {TEMP_WATCH_TABLE1}.new_repo_id = {TEMP_WATCH_TABLE3}.new_repo_id
        AND {TEMP_WATCH_TABLE3}.repo_id = {TEMP_WATCH_TABLE2}.repo_id
        ORDER BY watch_rank'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE5,
    TEMP_WATCH_TABLE2=TEMP_TABLE_NAME8_3,
    TEMP_WATCH_TABLE3=TEMP_WATCH_TABLE2,
    TEMP_WATCH_TABLE4=TEMP_WATCH_TABLE6
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [88]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE6))
data.head(10)

,new_repo_id,repo_name,watch_rank
0,6286308,996icu/996.ICU,1
1,312191,freeCodeCamp/freeCodeCamp,2
2,1193927,jwasham/coding-interview-university,3
3,128100,vuejs/vue,4
4,2403462,kamranahmedse/developer-roadmap,5
5,145820,EbookFoundation/free-programming-books,6
6,900570,public-apis/public-apis,7
7,113507,facebook/react,8
8,556665,tensorflow/tensorflow,9
9,236321,sindresorhus/awesome,10


### followers

#### 从TEMP_TABLE_NAME0中取出actor_id---actor_name对应表

In [89]:
TEMP_FOLLOW_TABLE0 = 'hx_repo_actor_birank_pull_forfollower_actorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id     BIGINT,
        actor_name    STRING
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519020507956gqso7yor2>

In [90]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT DISTINCT actor_id, actor_login
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY actor_id'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_TABLE_NAME0,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE0)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [91]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE0))
data.head(10)

,actor_id,actor_name
0,1,mojombo
1,2,defunkt
2,4,wycats
3,6,ivey
4,7,evanphx
5,17,vanpelt
6,18,wayneeseguin
7,19,brynary
8,20,kevinclark
9,21,technoweenie


In [92]:
# 39191的name
select_39191 = '''SELECT * FROM {TEMP_FOLLOW_TABLE} WHERE actor_id = 39191'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE0)
table_39191 = o.execute_sql(select_39191, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_39191.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record)

odps.Record {
  actor_id            39191
  actor_name          'paulirish'
}


#### 从ods_github_users中获得actor_id---actor_name唯一对应表

In [237]:
TEMP_FOLLOW_TABLE1 = 'hx_repo_actor_birank_pull_actorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_name    STRING,
        follower_cnt    BIGINT DEFAULT 0,
        time    DATE
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519110713137gvz28rsa>

In [238]:
follower_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE1}
        SELECT login, CHAR_MATCHCOUNT(followers, ',') + 1 AS follower_cnt, lastupdatedat
        FROM ods_github_users
        WHERE login NOT LIKE '%bot%' AND TO_CHAR(lastupdatedat,'yyyymmdd') >= '20160101'
        AND login in (SELECT actor_name FROM {TEMP_FOLLOW_TABLE0})
        ORDER BY follower_cnt desc
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE0,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE1
)
table = o.execute_sql(follower_sql, hints={"odps.sql.submit.mode" : "script"})

In [239]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE1))
data.head(10)

,actor_name,follower_cnt,time
0,torvalds,148982,2022-01-02
1,torvalds,146716,2022-01-02
2,yyx990803,77598,2022-01-24
3,yyx990803,76771,2022-01-24
4,yyx990803,75801,2022-01-24
5,gaearon,71341,2022-01-13
6,gaearon,70503,2022-01-13
7,gaearon,69646,2022-01-13
8,ruanyf,69357,2022-01-21
9,ruanyf,68883,2022-01-21


#### 将actor_name以actor_id代替，得到actor_id---follower_cnt---time对应表

In [240]:
TEMP_FOLLOW_TABLE1_00 = 'hx_repo_actor_birank_pull_id_to_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        follower_cnt    BIGINT,
        time    DATE
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE1_00
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519110735966gxg9akm7>

In [241]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT actor_id, follower_cnt, time
        FROM {TEMP_FOLLOW_TABLE0}, {TEMP_FOLLOW_TABLE1}
        WHERE {TEMP_FOLLOW_TABLE0}.actor_name = {TEMP_FOLLOW_TABLE1}.actor_name
        ORDER BY follower_cnt DESC
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE0,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE1,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE1_00
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [242]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE1_00))
data.head(10)

,actor_id,follower_cnt,time
0,1024025,148982,2022-01-02
1,1024025,146716,2022-01-02
2,499550,77598,2022-01-24
3,499550,76771,2022-01-24
4,499550,75801,2022-01-24
5,810438,71341,2022-01-13
6,810438,70503,2022-01-13
7,810438,69646,2022-01-13
8,905434,69357,2022-01-21
9,905434,68883,2022-01-21


#### 获得最新的actor_id对应的follow_cnt

In [243]:
TEMP_FOLLOW_TABLE1_0 = 'hx_repo_actor_birank_pull_today_newest'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        follow_cnt    BIGINT,
        time    DATE
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE1_0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519110750870gqu4yxqw2>

In [244]:
today_newest_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE1}
        SELECT actor_id, MAX(follower_cnt) AS follow_cnt, time
        FROM {TEMP_FOLLOW_TABLE0}
        GROUP BY actor_id, time
        ORDER BY follow_cnt DESC
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE1_00,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE1_0
)
table = o.execute_sql(today_newest_sql, hints={"odps.sql.submit.mode" : "script"})

In [245]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE1_0))
data.head(10)

,actor_id,follow_cnt,time
0,1024025,148982,2022-01-02
1,499550,77598,2022-01-24
2,810438,71341,2022-01-13
3,905434,69357,2022-01-21
4,66577,61869,2022-01-22
5,170270,46660,2022-01-29
6,470058,34435,2022-01-19
7,150330,33945,2022-01-28
8,39191,30116,2021-12-27
9,8547538,27087,2022-01-17


#### 仅保留actor_id---follow_cnt信息

In [246]:
TEMP_FOLLOW_TABLE2 = 'hx_repo_actor_birank_pull_latest_time'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        follow_cnt    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519110801935gyu92i5s2>

In [247]:
latest_time_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE1}
        SELECT actor_id, follow_cnt
        FROM {TEMP_FOLLOW_TABLE0}
        ORDER BY follow_cnt DESC
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE1_0,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE2
)
table = o.execute_sql(latest_time_sql, hints={"odps.sql.submit.mode" : "script"})

In [248]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE2))
data.head(10)

,actor_id,follow_cnt
0,1024025,148982
1,499550,77598
2,810438,71341
3,905434,69357
4,66577,61869
5,170270,46660
6,470058,34435
7,150330,33945
8,39191,30116
9,8547538,27087


#### 建立actor_id---new_actor_id唯一对应关系

In [250]:
TEMP_FOLLOW_TABLE3_0 = 'hx_repo_actor_birank_only_actor_id'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3_0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519110830975gsjzkkm7>

In [251]:
select_only_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT DISTINCT actor_id
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY actor_id
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE0,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE3_0
)
table = o.execute_sql(select_only_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [252]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE3_0))
data.head(10)

,actor_id
0,1
1,2
2,4
3,6
4,7
5,17
6,18
7,19
8,20
9,21


In [253]:
TEMP_FOLLOW_TABLE3 = 'hx_repo_actor_birank_new_id'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        new_actor_id    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519110849849gkjkzjm7>

In [254]:
insert_new_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT actor_id, row_number() over() - 1 as new_actor_id
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY actor_id
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE3_0,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE3
)
table = o.execute_sql(insert_new_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [255]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE3))
data.head(10)

,actor_id,new_actor_id
0,1,0
1,2,1
2,4,2
3,6,3
4,7,4
5,17,5
6,18,6
7,19,7
8,20,8
9,21,9


In [256]:
# 查看actor_id=39191对应的new_actor_id
select_39191 = '''SELECT * FROM {TEMP_FOLLOW_TABLE} WHERE new_actor_id = 1389704'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3)
table_39191 = o.execute_sql(select_39191, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_39191.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record)

odps.Record {
  actor_id                10810283
  new_actor_id            1389704
}


In [257]:
# 查看最大new_actor_id(用于保证与算法中的new_actor_id一致)
select_max_new_actor_id = '''SELECT MAX(new_actor_id) FROM {TEMP_FOLLOW_TABLE}'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3)
table_max_new_actor_id = o.execute_sql(select_max_new_actor_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_actor_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   7282527
}


#### 建立new_actor_id---follower_cnt对应关系

In [258]:
TEMP_FOLLOW_TABLE4 = 'hx_repo_actor_birank_follower_table_new'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        new_actor_id    BIGINT,
        follower_cnt    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519110948600gc06w5pr2>

In [260]:
newid_follow_cnt_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE3}
        SELECT new_actor_id, follow_cnt
        FROM {TEMP_FOLLOW_TABLE1}, {TEMP_FOLLOW_TABLE2}
        WHERE {TEMP_FOLLOW_TABLE1}.actor_id = {TEMP_FOLLOW_TABLE2}.actor_id
        ORDER BY follow_cnt desc
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE2,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE3,
    TEMP_FOLLOW_TABLE3=TEMP_FOLLOW_TABLE4
)
table = o.execute_sql(newid_follow_cnt_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [261]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE4))
data.head(10)

,new_actor_id,follower_cnt
0,255465,148982
1,141340,77598
2,210066,71341
3,230828,69357
4,27835,61869
5,61385,46660
6,134490,34435
7,55240,33945
8,17582,30116
9,1175929,27087


#### 得到new_actor_id---follower_rank对应关系

In [262]:
TEMP_FOLLOW_TABLE5 = 'hx_repo_actor_birank_follower_rank'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        new_actor_id    BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519111024284g3cdd9o7>

In [263]:
insert_rank_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT new_actor_id, row_number() over() AS rank
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY rank
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE4,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE5
)
table = o.execute_sql(insert_rank_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [264]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE5))
data.head(10)

,new_actor_id,follower_rank
0,255465,1
1,141340,2
2,210066,3
3,230828,4
4,27835,5
5,61385,6
6,134490,7
7,55240,8
8,17582,9
9,1175929,10


In [265]:
select_39191 = '''SELECT MAX(new_actor_id) FROM {TEMP_FOLLOW_TABLE}'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE5)
table_39191 = o.execute_sql(select_39191, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_39191.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record)

odps.Record {
  _c0   7280595
}


In [266]:
pd_idfollow = data.to_pandas()
pd_idfollow.head()

,new_actor_id,follower_rank
0,255465,1
1,141340,2
2,210066,3
3,230828,4
4,27835,5


In [267]:
follow_10 = pd_idfollow[:10]
follow_10

,new_actor_id,follower_rank
0,255465,1
1,141340,2
2,210066,3
3,230828,4
4,27835,5
5,61385,6
6,134490,7
7,55240,8
8,17582,9
9,1175929,10


In [268]:
follow_20 = pd_idfollow[:20]
follow_20

,new_actor_id,follower_rank
0,255465,1
1,141340,2
2,210066,3
3,230828,4
4,27835,5
5,61385,6
6,134490,7
7,55240,8
8,17582,9
9,1175929,10


In [269]:
follow_50 = pd_idfollow[:50]
follow_50

,new_actor_id,follower_rank
0,255465,1
1,141340,2
2,210066,3
3,230828,4
4,27835,5
5,61385,6
6,134490,7
7,55240,8
8,17582,9
9,1175929,10


#### 为便于展示，建立new_actor_id---actor_name---rank对应关系表

由于用户可能改名，actor_name可能会改变，但actor_id是不变的。所以，选取时间最近的actor_id对应的actor_name作为唯一对应的用户名，以便在github查询该用户名时能够得到相应页面

In [270]:
TEMP_FOLLOW_TABLE7 = 'hx_repo_actor_birank_pull_follower_actor_newid_and_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        new_actor_id     BIGINT,
        actor_name    STRING,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE7
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519111116595gxyr7om7>

In [271]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE4}
        SELECT
            {TEMP_FOLLOW_TABLE1}.new_actor_id,
            actor_name,
            follower_rank
        FROM {TEMP_FOLLOW_TABLE1}, {TEMP_FOLLOW_TABLE2}, {TEMP_FOLLOW_TABLE3}
        WHERE {TEMP_FOLLOW_TABLE1}.new_actor_id = {TEMP_FOLLOW_TABLE3}.new_actor_id
        AND {TEMP_FOLLOW_TABLE3}.actor_id = {TEMP_FOLLOW_TABLE2}.actor_id
        ORDER BY follower_rank'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_FOLLOW_TABLE2=TEMP_TABLE_NAME9_3,
    TEMP_FOLLOW_TABLE3=TEMP_FOLLOW_TABLE3,
    TEMP_FOLLOW_TABLE4=TEMP_FOLLOW_TABLE7
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [272]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE7))
data.head(10)

,new_actor_id,actor_name,follower_rank
0,255465,torvalds,1
1,141340,yyx990803,2
2,210066,gaearon,3
3,230828,ruanyf,4
4,27835,JakeWharton,5
5,61385,sindresorhus,6
6,134490,michaelliao,7
7,55240,getify,8
8,17582,paulirish,9
9,1175929,buckyroberts,10


## 评价

### birank

#### birank表前10/20/50的repo在watch中对应排名

##### 前10

In [159]:
birank_repo_top10=repo_info['new_repo_id'][:10].tolist()
print(birank_repo_top10)

[1614471, 170062, 8262113, 17203, 3854851, 6432194, 2538677, 267844, 71898, 1136011]


In [160]:
TEMP_EVAL_TABLE1 = 'hx_birank_watch_repo_top10'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519063908187g9b5i592>

In [161]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE1,
    id1=birank_repo_top10[0],
    id2=birank_repo_top10[1],
    id3=birank_repo_top10[2],
    id4=birank_repo_top10[3],
    id5=birank_repo_top10[4],
    id6=birank_repo_top10[5],
    id7=birank_repo_top10[6],
    id8=birank_repo_top10[7],
    id9=birank_repo_top10[8],
    id10=birank_repo_top10[9],
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [162]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE1))
data.head(10)

,new_repo_id,watch_rank
0,71898,236
1,1614471,437
2,6432194,3459
3,17203,9801
4,2538677,12057
5,3854851,16991
6,170062,25118
7,267844,26833
8,1136011,32220
9,8262113,53243


In [163]:
birank_repo_top10_inwatch = data.to_pandas()
birank_repo_top10_inwatch

,new_repo_id,watch_rank
0,71898,236
1,1614471,437
2,6432194,3459
3,17203,9801
4,2538677,12057
5,3854851,16991
6,170062,25118
7,267844,26833
8,1136011,32220
9,8262113,53243


In [164]:
repo_result_top10 = pd.DataFrame(repo_result, columns=['new_repo_id','repo_name','rank'])[:10]
repo_result_top10

,new_repo_id,repo_name,rank
0,1614471,firstcontributions/first-contributions,1
1,170062,jlord/patchwork,2
2,8262113,google/it-cert-automation-practice,3
3,17203,octocat/Spoon-Knife,4
4,3854851,zero-to-mastery/start-here-guidelines,5
5,6432194,freddier/hyperblog,6
6,2538677,udacity/course-collaboration-travel-plans,7
7,267844,JetBrains/swot,8
8,71898,DefinitelyTyped/DefinitelyTyped,9
9,1136011,githubschool/open-enrollment-classes-introduct...,10


In [165]:
# 合并birank_repo_top10_inwatch和repo_result_top10
birank_watch_top10 = pd.merge(repo_result_top10, birank_repo_top10_inwatch, on='new_repo_id')
birank_watch_top10

,new_repo_id,repo_name,rank,watch_rank
0,1614471,firstcontributions/first-contributions,1,437
1,170062,jlord/patchwork,2,25118
2,8262113,google/it-cert-automation-practice,3,53243
3,17203,octocat/Spoon-Knife,4,9801
4,3854851,zero-to-mastery/start-here-guidelines,5,16991
5,6432194,freddier/hyperblog,6,3459
6,2538677,udacity/course-collaboration-travel-plans,7,12057
7,267844,JetBrains/swot,8,26833
8,71898,DefinitelyTyped/DefinitelyTyped,9,236
9,1136011,githubschool/open-enrollment-classes-introduct...,10,32220


##### 前20

In [166]:
birank_repo_top20=repo_info['new_repo_id'][:20].tolist()
print(birank_repo_top20)

[1614471, 170062, 8262113, 17203, 3854851, 6432194, 2538677, 267844, 71898, 1136011, 7159897, 267651, 12790580, 1812687, 11255453, 5817026, 117139, 171578, 6267008, 312191]


In [167]:
TEMP_EVAL_TABLE2 = 'hx_birank_watch_repo_top20'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519063924972gin4pta2>

In [168]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10} OR new_repo_id = {id11} OR new_repo_id = {id12}
        OR new_repo_id = {id13} OR new_repo_id = {id14} OR new_repo_id = {id15} OR new_repo_id = {id16}
        OR new_repo_id = {id17} OR new_repo_id = {id18} OR new_repo_id = {id19} OR new_repo_id = {id20}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE2,
    id1=birank_repo_top20[0],
    id2=birank_repo_top20[1],
    id3=birank_repo_top20[2],
    id4=birank_repo_top20[3],
    id5=birank_repo_top20[4],
    id6=birank_repo_top20[5],
    id7=birank_repo_top20[6],
    id8=birank_repo_top20[7],
    id9=birank_repo_top20[8],
    id10=birank_repo_top20[9],
    id11=birank_repo_top20[10],
    id12=birank_repo_top20[11],
    id13=birank_repo_top20[12],
    id14=birank_repo_top20[13],
    id15=birank_repo_top20[14],
    id16=birank_repo_top20[15],
    id17=birank_repo_top20[16],
    id18=birank_repo_top20[17],
    id19=birank_repo_top20[18],
    id20=birank_repo_top20[19]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [169]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE2))
data.head(10)

,new_repo_id,watch_rank
0,312191,2
1,71898,236
2,1614471,437
3,1812687,3373
4,6432194,3459
5,117139,5952
6,17203,9801
7,2538677,12057
8,7159897,14580
9,3854851,16991


In [170]:
birank_repo_top20_inwatch = data.to_pandas()
repo_result_top20 = pd.DataFrame(repo_result, columns=['new_repo_id','repo_name','rank'])[:20]
# 合并birank_repo_top20_inwatch和repo_result_top20
birank_watch_top20 = pd.merge(repo_result_top20, birank_repo_top20_inwatch, on='new_repo_id')
birank_watch_top20

,new_repo_id,repo_name,rank,watch_rank
0,1614471,firstcontributions/first-contributions,1,437
1,170062,jlord/patchwork,2,25118
2,8262113,google/it-cert-automation-practice,3,53243
3,17203,octocat/Spoon-Knife,4,9801
4,3854851,zero-to-mastery/start-here-guidelines,5,16991
5,6432194,freddier/hyperblog,6,3459
6,2538677,udacity/course-collaboration-travel-plans,7,12057
7,267844,JetBrains/swot,8,26833
8,71898,DefinitelyTyped/DefinitelyTyped,9,236
9,1136011,githubschool/open-enrollment-classes-introduct...,10,32220


##### 前50

In [171]:
birank_repo_top50=repo_info['new_repo_id'][:50].tolist()
print(birank_repo_top50)

[1614471, 170062, 8262113, 17203, 3854851, 6432194, 2538677, 267844, 71898, 1136011, 7159897, 267651, 12790580, 1812687, 11255453, 5817026, 117139, 171578, 6267008, 312191, 12506950, 2244040, 221282, 5630121, 2889501, 45413, 3350101, 13004353, 270750, 216995, 45595, 820970, 13621948, 4348406, 4529304, 556665, 310846, 745600, 407886, 133512, 7553599, 86081, 577135, 6428248, 371099, 8918894, 262607, 9613777, 139854, 1018077]


In [172]:
TEMP_EVAL_TABLE3 = 'hx_birank_watch_repo_top50'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519063931191g9lp7yor2>

In [173]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10} OR new_repo_id = {id11} OR new_repo_id = {id12}
        OR new_repo_id = {id13} OR new_repo_id = {id14} OR new_repo_id = {id15} OR new_repo_id = {id16}
        OR new_repo_id = {id17} OR new_repo_id = {id18} OR new_repo_id = {id19} OR new_repo_id = {id20}
        OR new_repo_id = {id21} OR new_repo_id = {id22} OR new_repo_id = {id23} OR new_repo_id = {id24}
        OR new_repo_id = {id25} OR new_repo_id = {id26} OR new_repo_id = {id27} OR new_repo_id = {id28}
        OR new_repo_id = {id29} OR new_repo_id = {id30} OR new_repo_id = {id31} OR new_repo_id = {id32}
        OR new_repo_id = {id33} OR new_repo_id = {id34} OR new_repo_id = {id35} OR new_repo_id = {id36}
        OR new_repo_id = {id37} OR new_repo_id = {id38} OR new_repo_id = {id39} OR new_repo_id = {id40}
        OR new_repo_id = {id41} OR new_repo_id = {id42} OR new_repo_id = {id43} OR new_repo_id = {id44}
        OR new_repo_id = {id45} OR new_repo_id = {id46} OR new_repo_id = {id47} OR new_repo_id = {id48}
        OR new_repo_id = {id49} OR new_repo_id = {id50}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE3,
    id1=birank_repo_top50[0], id2=birank_repo_top50[1], id3=birank_repo_top50[2], id4=birank_repo_top50[3],
    id5=birank_repo_top50[4], id6=birank_repo_top50[5], id7=birank_repo_top50[6], id8=birank_repo_top50[7],
    id9=birank_repo_top50[8], id10=birank_repo_top50[9], id11=birank_repo_top50[10], id12=birank_repo_top50[11],
    id13=birank_repo_top50[12], id14=birank_repo_top50[13], id15=birank_repo_top50[14], id16=birank_repo_top50[15],
    id17=birank_repo_top50[16], id18=birank_repo_top50[17], id19=birank_repo_top50[18], id20=birank_repo_top50[19],
    id21=birank_repo_top50[20], id22=birank_repo_top50[21], id23=birank_repo_top50[22], id24=birank_repo_top50[23],
    id25=birank_repo_top50[24], id26=birank_repo_top50[25], id27=birank_repo_top50[26], id28=birank_repo_top50[27],
    id29=birank_repo_top50[28], id30=birank_repo_top50[29], id31=birank_repo_top50[30], id32=birank_repo_top50[31],
    id33=birank_repo_top50[32], id34=birank_repo_top50[33], id35=birank_repo_top50[34], id36=birank_repo_top50[35],
    id37=birank_repo_top50[36], id38=birank_repo_top50[37], id39=birank_repo_top50[38], id40=birank_repo_top50[39],
    id41=birank_repo_top50[40], id42=birank_repo_top50[41], id43=birank_repo_top50[42], id44=birank_repo_top50[43],
    id45=birank_repo_top50[44], id46=birank_repo_top50[45], id47=birank_repo_top50[46], id48=birank_repo_top50[47],
    id49=birank_repo_top50[48], id50=birank_repo_top50[49]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [174]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE3))
data.head(10)

,new_repo_id,watch_rank
0,312191,2
1,556665,9
2,407886,89
3,139854,105
4,45595,161
5,71898,236
6,1614471,437
7,86081,487
8,577135,939
9,45413,1493


In [175]:
birank_repo_top50_inwatch = data.to_pandas()
repo_result_top50 = pd.DataFrame(repo_result, columns=['new_repo_id','repo_name','rank'])[:50]
# 合并birank_repo_top50_inwatch和repo_result_top50
birank_watch_top50 = pd.merge(repo_result_top50, birank_repo_top50_inwatch, on='new_repo_id')
birank_watch_top50

,new_repo_id,repo_name,rank,watch_rank
0,1614471,firstcontributions/first-contributions,1,437
1,170062,jlord/patchwork,2,25118
2,8262113,google/it-cert-automation-practice,3,53243
3,17203,octocat/Spoon-Knife,4,9801
4,3854851,zero-to-mastery/start-here-guidelines,5,16991
5,6432194,freddier/hyperblog,6,3459
6,2538677,udacity/course-collaboration-travel-plans,7,12057
7,267844,JetBrains/swot,8,26833
8,71898,DefinitelyTyped/DefinitelyTyped,9,236
9,1136011,githubschool/open-enrollment-classes-introduct...,10,32220


#### birank中排名前10/20/50的actor在follower中的排名

##### top10

In [176]:
birank_actor_top10=actor_info['new_actor_id'][:10].tolist()
print(birank_actor_top10)

[1389704, 1065416, 5631549, 2730410, 1172471, 2263149, 4823957, 2121899, 5148049, 6575172]


In [273]:
TEMP_EVAL_TABLE4 = 'hx_birank_follow_actor_top10'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519111147549gbbs7om7>

In [274]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE4,
    id1=birank_actor_top10[0],
    id2=birank_actor_top10[1],
    id3=birank_actor_top10[2],
    id4=birank_actor_top10[3],
    id5=birank_actor_top10[4],
    id6=birank_actor_top10[5],
    id7=birank_actor_top10[6],
    id8=birank_actor_top10[7],
    id9=birank_actor_top10[8],
    id10=birank_actor_top10[9]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [286]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE4))
data.head(10)

,new_actor_id,follower_rank
0,1172471,4982
1,2730410,19847
2,1389704,128125
3,1065416,128816
4,2263149,159060
5,5148049,173762
6,6575172,180074
7,2121899,184369


In [287]:
birank_actor_top10_infollow = data.to_pandas()
actor_result_top10 = pd.DataFrame(actor_result, columns=['new_actor_id','actor_name','rank'])[:10]
# 合并birank_actor_top10_infollow和actor_result_top10
birank_follow_top10 = pd.merge(actor_result_top10, birank_actor_top10_infollow, on='new_actor_id')
birank_follow_top10

,new_actor_id,actor_name,rank,follower_rank
0,1389704,direwolf-github,1,128125
1,1065416,xmo-odoo,2,128816
2,2730410,codacy-badger,4,19847
3,1172471,gitter-badger,5,4982
4,2263149,QualitySoftwareDeveloper,6,159060
5,2121899,idsb3t1,8,184369
6,5148049,codeserver-service-qa,9,173762
7,6575172,turbocanary-admin,10,180074


##### top20

In [224]:
birank_actor_top20=actor_info['new_actor_id'][:20].tolist()
print(birank_actor_top20)

[1389704, 1065416, 5631549, 2730410, 1172471, 2263149, 4823957, 2121899, 5148049, 6575172, 1103564, 6270054, 6282961, 292688, 318510, 4357576, 4087589, 58376, 5580835, 1100073]


In [277]:
TEMP_EVAL_TABLE5 = 'hx_birank_follow_actor_top20'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519111207900grls7om7>

In [278]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10} OR new_actor_id = {id11} OR new_actor_id = {id12}
        OR new_actor_id = {id13} OR new_actor_id = {id14} OR new_actor_id = {id15} OR new_actor_id = {id16}
        OR new_actor_id = {id17} OR new_actor_id = {id18} OR new_actor_id = {id19} OR new_actor_id = {id20}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE5,
    id1=birank_actor_top20[0],
    id2=birank_actor_top20[1],
    id3=birank_actor_top20[2],
    id4=birank_actor_top20[3],
    id5=birank_actor_top20[4],
    id6=birank_actor_top20[5],
    id7=birank_actor_top20[6],
    id8=birank_actor_top20[7],
    id9=birank_actor_top20[8],
    id10=birank_actor_top20[9],
    id11=birank_actor_top20[10],
    id12=birank_actor_top20[11],
    id13=birank_actor_top20[12],
    id14=birank_actor_top20[13],
    id15=birank_actor_top20[14],
    id16=birank_actor_top20[15],
    id17=birank_actor_top20[16],
    id18=birank_actor_top20[17],
    id19=birank_actor_top20[18],
    id20=birank_actor_top20[19]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [284]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE5))
data.head()

,new_actor_id,follower_rank
0,318510,3452
1,1172471,4982
2,2730410,19847
3,292688,61054
4,1389704,128125
5,1065416,128816
6,1103564,152899
7,4357576,154295
8,2263149,159060
9,4087589,170586


In [285]:
birank_actor_top20_infollow = data.to_pandas()
actor_result_top20 = pd.DataFrame(actor_result, columns=['new_actor_id','actor_name','rank'])[:20]
# 合并birank_actor_top20_infollow和actor_result_top20
birank_follow_top20 = pd.merge(actor_result_top20, birank_actor_top20_infollow, on='new_actor_id')
birank_follow_top20

,new_actor_id,actor_name,rank,follower_rank
0,1389704,direwolf-github,1,128125
1,1065416,xmo-odoo,2,128816
2,2730410,codacy-badger,4,19847
3,1172471,gitter-badger,5,4982
4,2263149,QualitySoftwareDeveloper,6,159060
5,2121899,idsb3t1,8,184369
6,5148049,codeserver-service-qa,9,173762
7,6575172,turbocanary-admin,10,180074
8,1103564,slskopytko,11,152899
9,6270054,shiftleft-staging,12,174154


##### top50

In [230]:
birank_actor_top50=actor_info['new_actor_id'][:50].tolist()
print(birank_actor_top50)

[1389704, 1065416, 5631549, 2730410, 1172471, 2263149, 4823957, 2121899, 5148049, 6575172, 1103564, 6270054, 6282961, 292688, 318510, 4357576, 4087589, 58376, 5580835, 1100073, 454546, 989410, 640086, 1178804, 6806159, 6185714, 4074427, 15183, 140539, 618299, 5177482, 4439111, 5074948, 3693443, 477752, 492387, 492993, 528756, 4867185, 3043122, 3703474, 5710996, 1799194, 5829576, 3696396, 4883463, 908745, 2675763, 4487914, 951365]


In [281]:
TEMP_EVAL_TABLE6 = 'hx_birank_follow_actor_top50'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE6
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519111249505ggonkom7>

In [282]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10} OR new_actor_id = {id11} OR new_actor_id = {id12}
        OR new_actor_id = {id13} OR new_actor_id = {id14} OR new_actor_id = {id15} OR new_actor_id = {id16}
        OR new_actor_id = {id17} OR new_actor_id = {id18} OR new_actor_id = {id19} OR new_actor_id = {id20}
        OR new_actor_id = {id21} OR new_actor_id = {id22} OR new_actor_id = {id23} OR new_actor_id = {id24}
        OR new_actor_id = {id25} OR new_actor_id = {id26} OR new_actor_id = {id27} OR new_actor_id = {id28}
        OR new_actor_id = {id29} OR new_actor_id = {id30} OR new_actor_id = {id31} OR new_actor_id = {id32}
        OR new_actor_id = {id33} OR new_actor_id = {id34} OR new_actor_id = {id35} OR new_actor_id = {id36}
        OR new_actor_id = {id37} OR new_actor_id = {id38} OR new_actor_id = {id39} OR new_actor_id = {id40}
        OR new_actor_id = {id41} OR new_actor_id = {id42} OR new_actor_id = {id43} OR new_actor_id = {id44}
        OR new_actor_id = {id45} OR new_actor_id = {id46} OR new_actor_id = {id47} OR new_actor_id = {id48}
        OR new_actor_id = {id49} OR new_actor_id = {id50}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE6,
    id1=birank_actor_top50[0], id2=birank_actor_top50[1], id3=birank_actor_top50[2], id4=birank_actor_top50[3],
    id5=birank_actor_top50[4], id6=birank_actor_top50[5], id7=birank_actor_top50[6], id8=birank_actor_top50[7],
    id9=birank_actor_top50[8], id10=birank_actor_top50[9], id11=birank_actor_top50[10], id12=birank_actor_top50[11],
    id13=birank_actor_top50[12], id14=birank_actor_top50[13], id15=birank_actor_top50[14], id16=birank_actor_top50[15],
    id17=birank_actor_top50[16], id18=birank_actor_top50[17], id19=birank_actor_top50[18], id20=birank_actor_top50[19],
    id21=birank_actor_top50[20], id22=birank_actor_top50[21], id23=birank_actor_top50[22], id24=birank_actor_top50[23],
    id25=birank_actor_top50[24], id26=birank_actor_top50[25], id27=birank_actor_top50[26], id28=birank_actor_top50[27],
    id29=birank_actor_top50[28], id30=birank_actor_top50[29], id31=birank_actor_top50[30], id32=birank_actor_top50[31],
    id33=birank_actor_top50[32], id34=birank_actor_top50[33], id35=birank_actor_top50[34], id36=birank_actor_top50[35],
    id37=birank_actor_top50[36], id38=birank_actor_top50[37], id39=birank_actor_top50[38], id40=birank_actor_top50[39],
    id41=birank_actor_top50[40], id42=birank_actor_top50[41], id43=birank_actor_top50[42], id44=birank_actor_top50[43],
    id45=birank_actor_top50[44], id46=birank_actor_top50[45], id47=birank_actor_top50[46], id48=birank_actor_top50[47],
    id49=birank_actor_top50[48], id50=birank_actor_top50[49]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [288]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE6))
data.head()

,new_actor_id,follower_rank
0,618299,564
1,454546,810
2,318510,3452
3,492387,4228
4,640086,4501
5,1172471,4982
6,1178804,5741
7,3043122,9874
8,951365,12415
9,15183,12549


In [289]:
birank_actor_top50_infollow = data.to_pandas()
actor_result_top50 = pd.DataFrame(actor_result, columns=['new_actor_id','actor_name','rank'])[:50]
# 合并birank_actor_top50_infollow和actor_result_top50
birank_follow_top50 = pd.merge(actor_result_top50, birank_actor_top50_infollow, on='new_actor_id')
birank_follow_top50

,new_actor_id,actor_name,rank,follower_rank
0,1389704,direwolf-github,1,128125
1,1065416,xmo-odoo,2,128816
2,2730410,codacy-badger,4,19847
3,1172471,gitter-badger,5,4982
4,2263149,QualitySoftwareDeveloper,6,159060
5,2121899,idsb3t1,8,184369
6,5148049,codeserver-service-qa,9,173762
7,6575172,turbocanary-admin,10,180074
8,1103564,slskopytko,11,152899
9,6270054,shiftleft-staging,12,174154


#### correlation

##### repo

birank_watch_top10, birank_watch_top20, birank_watch_top50中，rank与watch_rank的spearman相关系数检验

In [158]:
import scipy.stats
print("birank表前10的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(birank_watch_top10['rank'].to_list(), birank_watch_top10['watch_rank'].to_list())[0])
print("birank表前20的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(birank_watch_top20['rank'].to_list(), birank_watch_top20['watch_rank'].to_list())[0])
print("birank表前50的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(birank_watch_top50['rank'].to_list(), birank_watch_top50['watch_rank'].to_list())[0])

birank表前10的repo的排名与其在watch中对应排名的correlation值： 0.06666666666666665
birank表前20的repo的排名与其在watch中对应排名的correlation值： 0.18045112781954886
birank表前50的repo的排名与其在watch中对应排名的correlation值： 0.10569027611044417


由于correlation在(0, 0.2)之间，故相关性弱。

birank得到的排名靠前的repo排名与watch统计信息中的排名几乎不相关，二者结果相差较大

##### actor

In [292]:
print("birank表前10的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(birank_follow_top10['rank'].to_list(), birank_follow_top10['follower_rank'].to_list())[0])
print("birank表前20的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(birank_follow_top20['rank'].to_list(), birank_follow_top20['follower_rank'].to_list())[0])
print("birank表前50的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(birank_follow_top50['rank'].to_list(), birank_follow_top50['follower_rank'].to_list())[0])

birank表前10的actor的排名与其在follow中对应排名的correlation值： 0.7142857142857144
birank表前20的actor的排名与其在follow中对应排名的correlation值： 0.3147058823529411
birank表前50的actor的排名与其在follow中对应排名的correlation值： -0.24461420932009165


与repo情况相似，birank得到的排名靠前的actor排名与follower统计信息中的排名几乎不相关

### pagerank

#### pagerank前10/20/50的repo在watch中的排名

##### top10

In [302]:
pagerank_repo_top10=repo_pagerank_info['new_repo_id'][:10].tolist()
print(pagerank_repo_top10)

[1614471, 170062, 8262113, 17203, 3854851, 71898, 2538677, 6432194, 267844, 1136011]


In [303]:
TEMP_EVAL_TABLE7 = 'hx_pagerank_watch_repo_top10'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE7
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519115842679ggciggr>

In [304]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE7,
    id1=pagerank_repo_top10[0],
    id2=pagerank_repo_top10[1],
    id3=pagerank_repo_top10[2],
    id4=pagerank_repo_top10[3],
    id5=pagerank_repo_top10[4],
    id6=pagerank_repo_top10[5],
    id7=pagerank_repo_top10[6],
    id8=pagerank_repo_top10[7],
    id9=pagerank_repo_top10[8],
    id10=pagerank_repo_top10[9]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [305]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE7))
data.head(10)

,new_repo_id,watch_rank
0,71898,236
1,1614471,437
2,6432194,3459
3,17203,9801
4,2538677,12057
5,3854851,16991
6,170062,25118
7,267844,26833
8,1136011,32220
9,8262113,53243


In [306]:
pagerank_repo_top10_inwatch = data.to_pandas()
repo_pagerank_result_top10 = pd.DataFrame(repo_pagerank_result, columns=['new_repo_id','repo_name','rank'])[:10]
# 合并pagerank_repo_top10_inwatch和repo_pagerank_result_top10
pagerank_watch_top10 = pd.merge(repo_pagerank_result_top10, pagerank_repo_top10_inwatch, on='new_repo_id')
pagerank_watch_top10

,new_repo_id,repo_name,rank,watch_rank
0,1614471,firstcontributions/first-contributions,1,437
1,170062,jlord/patchwork,2,25118
2,8262113,google/it-cert-automation-practice,3,53243
3,17203,octocat/Spoon-Knife,4,9801
4,3854851,zero-to-mastery/start-here-guidelines,5,16991
5,71898,DefinitelyTyped/DefinitelyTyped,6,236
6,2538677,udacity/course-collaboration-travel-plans,7,12057
7,6432194,freddier/hyperblog,8,3459
8,267844,JetBrains/swot,9,26833
9,1136011,githubschool/open-enrollment-classes-introduct...,10,32220


##### top20

In [309]:
pagerank_repo_top20=repo_pagerank_info['new_repo_id'][:20].tolist()
print(pagerank_repo_top20)

[1614471, 170062, 8262113, 17203, 3854851, 71898, 2538677, 6432194, 267844, 1136011, 12790580, 7159897, 267651, 1812687, 312191, 117139, 12506950, 45413, 3350101, 221282]


In [307]:
TEMP_EVAL_TABLE8 = 'hx_pagerank_watch_repo_top20'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE8
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022051912035086gdy1ta2a>

In [310]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10} OR new_repo_id = {id11} OR new_repo_id = {id12}
        OR new_repo_id = {id13} OR new_repo_id = {id14} OR new_repo_id = {id15} OR new_repo_id = {id16}
        OR new_repo_id = {id17} OR new_repo_id = {id18} OR new_repo_id = {id19} OR new_repo_id = {id20}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE8,
    id1=pagerank_repo_top20[0],
    id2=pagerank_repo_top20[1],
    id3=pagerank_repo_top20[2],
    id4=pagerank_repo_top20[3],
    id5=pagerank_repo_top20[4],
    id6=pagerank_repo_top20[5],
    id7=pagerank_repo_top20[6],
    id8=pagerank_repo_top20[7],
    id9=pagerank_repo_top20[8],
    id10=pagerank_repo_top20[9],
    id11=pagerank_repo_top20[10],
    id12=pagerank_repo_top20[11],
    id13=pagerank_repo_top20[12],
    id14=pagerank_repo_top20[13],
    id15=pagerank_repo_top20[14],
    id16=pagerank_repo_top20[15],
    id17=pagerank_repo_top20[16],
    id18=pagerank_repo_top20[17],
    id19=pagerank_repo_top20[18],
    id20=pagerank_repo_top20[19]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [311]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE8))
data.head()

,new_repo_id,watch_rank
0,312191,2
1,71898,236
2,1614471,437
3,45413,1493
4,1812687,3373
5,6432194,3459
6,117139,5952
7,17203,9801
8,12506950,9983
9,2538677,12057


In [312]:
pagerank_repo_top20_inwatch = data.to_pandas()
repo_pagerank_result_top20 = pd.DataFrame(repo_pagerank_result, columns=['new_repo_id','repo_name','rank'])[:20]
# 合并pagerank_repo_top20_inwatch和repo_pagerank_result_top20
pagerank_watch_top20 = pd.merge(repo_pagerank_result_top20, pagerank_repo_top20_inwatch, on='new_repo_id')
pagerank_watch_top20

,new_repo_id,repo_name,rank,watch_rank
0,1614471,firstcontributions/first-contributions,1,437
1,170062,jlord/patchwork,2,25118
2,8262113,google/it-cert-automation-practice,3,53243
3,17203,octocat/Spoon-Knife,4,9801
4,3854851,zero-to-mastery/start-here-guidelines,5,16991
5,71898,DefinitelyTyped/DefinitelyTyped,6,236
6,2538677,udacity/course-collaboration-travel-plans,7,12057
7,6432194,freddier/hyperblog,8,3459
8,267844,JetBrains/swot,9,26833
9,1136011,githubschool/open-enrollment-classes-introduct...,10,32220


##### top50

In [313]:
pagerank_repo_top50=repo_pagerank_info['new_repo_id'][:50].tolist()
print(pagerank_repo_top50)

[1614471, 170062, 8262113, 17203, 3854851, 71898, 2538677, 6432194, 267844, 1136011, 12790580, 7159897, 267651, 1812687, 312191, 117139, 12506950, 45413, 3350101, 221282, 11255453, 6267008, 5817026, 820970, 45595, 171578, 2244040, 5630121, 745600, 407886, 2889501, 577135, 556665, 310846, 270750, 86081, 56086, 216995, 223092, 7553599, 139854, 377161, 4348406, 1986850, 9613777, 13004353, 2770665, 13621948, 3315786, 304203]


In [314]:
TEMP_EVAL_TABLE9 = 'hx_pagerank_watch_repo_top50'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE9
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519120928677g4pr7om7>

In [315]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10} OR new_repo_id = {id11} OR new_repo_id = {id12}
        OR new_repo_id = {id13} OR new_repo_id = {id14} OR new_repo_id = {id15} OR new_repo_id = {id16}
        OR new_repo_id = {id17} OR new_repo_id = {id18} OR new_repo_id = {id19} OR new_repo_id = {id20}
        OR new_repo_id = {id21} OR new_repo_id = {id22} OR new_repo_id = {id23} OR new_repo_id = {id24}
        OR new_repo_id = {id25} OR new_repo_id = {id26} OR new_repo_id = {id27} OR new_repo_id = {id28}
        OR new_repo_id = {id29} OR new_repo_id = {id30} OR new_repo_id = {id31} OR new_repo_id = {id32}
        OR new_repo_id = {id33} OR new_repo_id = {id34} OR new_repo_id = {id35} OR new_repo_id = {id36}
        OR new_repo_id = {id37} OR new_repo_id = {id38} OR new_repo_id = {id39} OR new_repo_id = {id40}
        OR new_repo_id = {id41} OR new_repo_id = {id42} OR new_repo_id = {id43} OR new_repo_id = {id44}
        OR new_repo_id = {id45} OR new_repo_id = {id46} OR new_repo_id = {id47} OR new_repo_id = {id48}
        OR new_repo_id = {id49} OR new_repo_id = {id50}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE9,
    id1=pagerank_repo_top50[0], id2=pagerank_repo_top50[1], id3=pagerank_repo_top50[2], id4=pagerank_repo_top50[3],
    id5=pagerank_repo_top50[4], id6=pagerank_repo_top50[5], id7=pagerank_repo_top50[6], id8=pagerank_repo_top50[7],
    id9=pagerank_repo_top50[8], id10=pagerank_repo_top50[9], id11=pagerank_repo_top50[10], id12=pagerank_repo_top50[11],
    id13=pagerank_repo_top50[12], id14=pagerank_repo_top50[13], id15=pagerank_repo_top50[14], id16=pagerank_repo_top50[15],
    id17=pagerank_repo_top50[16], id18=pagerank_repo_top50[17], id19=pagerank_repo_top50[18], id20=pagerank_repo_top50[19],
    id21=pagerank_repo_top50[20], id22=pagerank_repo_top50[21], id23=pagerank_repo_top50[22], id24=pagerank_repo_top50[23],
    id25=pagerank_repo_top50[24], id26=pagerank_repo_top50[25], id27=pagerank_repo_top50[26], id28=pagerank_repo_top50[27],
    id29=pagerank_repo_top50[28], id30=pagerank_repo_top50[29], id31=pagerank_repo_top50[30], id32=pagerank_repo_top50[31],
    id33=pagerank_repo_top50[32], id34=pagerank_repo_top50[33], id35=pagerank_repo_top50[34], id36=pagerank_repo_top50[35],
    id37=pagerank_repo_top50[36], id38=pagerank_repo_top50[37], id39=pagerank_repo_top50[38], id40=pagerank_repo_top50[39],
    id41=pagerank_repo_top50[40], id42=pagerank_repo_top50[41], id43=pagerank_repo_top50[42], id44=pagerank_repo_top50[43],
    id45=pagerank_repo_top50[44], id46=pagerank_repo_top50[45], id47=pagerank_repo_top50[46], id48=pagerank_repo_top50[47],
    id49=pagerank_repo_top50[48], id50=pagerank_repo_top50[49]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [316]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE9))
data.head()

,new_repo_id,watch_rank
0,312191,2
1,556665,9
2,223092,45
3,407886,89
4,139854,105
5,45595,161
6,71898,236
7,1614471,437
8,86081,487
9,577135,939


In [317]:
pagerank_repo_top50_inwatch = data.to_pandas()
repo_pagerank_result_top50 = pd.DataFrame(repo_pagerank_result, columns=['new_repo_id','repo_name','rank'])[:50]
# 合并pagerank_repo_top50_inwatch和repo_pagerank_result_top50
pagerank_watch_top50 = pd.merge(repo_pagerank_result_top50, pagerank_repo_top50_inwatch, on='new_repo_id')
pagerank_watch_top50

,new_repo_id,repo_name,rank,watch_rank
0,1614471,firstcontributions/first-contributions,1,437
1,170062,jlord/patchwork,2,25118
2,8262113,google/it-cert-automation-practice,3,53243
3,17203,octocat/Spoon-Knife,4,9801
4,3854851,zero-to-mastery/start-here-guidelines,5,16991
5,71898,DefinitelyTyped/DefinitelyTyped,6,236
6,2538677,udacity/course-collaboration-travel-plans,7,12057
7,6432194,freddier/hyperblog,8,3459
8,267844,JetBrains/swot,9,26833
9,1136011,githubschool/open-enrollment-classes-introduct...,10,32220


#### pagerank前10/20/50的actor在follower中的排名

##### top10

In [318]:
pagerank_actor_top10=actor_pagerank_info['new_actor_id'][:10].tolist()
print(pagerank_actor_top10)

[1389704, 1065416, 5631549, 2730410, 1172471, 2263149, 4823957, 2121899, 5148049, 6575172]


In [320]:
TEMP_EVAL_TABLE10 = 'hx_pagerank_follow_actor_top10'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE10
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519122359778g0zf1ii5>

In [322]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE10,
    id1=pagerank_actor_top10[0],
    id2=pagerank_actor_top10[1],
    id3=pagerank_actor_top10[2],
    id4=pagerank_actor_top10[3],
    id5=pagerank_actor_top10[4],
    id6=pagerank_actor_top10[5],
    id7=pagerank_actor_top10[6],
    id8=pagerank_actor_top10[7],
    id9=pagerank_actor_top10[8],
    id10=pagerank_actor_top10[9]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [323]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE10))
data.head(10)

,new_actor_id,follower_rank
0,1172471,4982
1,2730410,19847
2,1389704,128125
3,1065416,128816
4,2263149,159060
5,5148049,173762
6,6575172,180074
7,2121899,184369


In [324]:
pagerank_actor_top10_infollow = data.to_pandas()
actor_pagerank_result_top10 = pd.DataFrame(actor_pagerank_result, columns=['new_actor_id','actor_name','rank'])[:10]
# 合并pagerank_actor_top10_infollow和actor_pagerank_result_top10
pagerank_follow_top10 = pd.merge(actor_pagerank_result_top10, pagerank_actor_top10_infollow, on='new_actor_id')
pagerank_follow_top10

,new_actor_id,actor_name,rank,follower_rank
0,1389704,direwolf-github,1,128125
1,1065416,xmo-odoo,2,128816
2,2730410,codacy-badger,4,19847
3,1172471,gitter-badger,5,4982
4,2263149,QualitySoftwareDeveloper,6,159060
5,2121899,idsb3t1,8,184369
6,5148049,codeserver-service-qa,9,173762
7,6575172,turbocanary-admin,10,180074


##### top20

In [325]:
pagerank_actor_top20=actor_pagerank_info['new_actor_id'][:20].tolist()
print(pagerank_actor_top20)

[1389704, 1065416, 5631549, 2730410, 1172471, 2263149, 4823957, 2121899, 5148049, 6575172, 1103564, 6270054, 6282961, 292688, 4357576, 4087589, 58376, 5580835, 318510, 6806159]


In [326]:
TEMP_EVAL_TABLE11 = 'hx_pagerank_follow_actor_top20'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE11
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519123149502gs174ssa>

In [327]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10} OR new_actor_id = {id11} OR new_actor_id = {id12}
        OR new_actor_id = {id13} OR new_actor_id = {id14} OR new_actor_id = {id15} OR new_actor_id = {id16}
        OR new_actor_id = {id17} OR new_actor_id = {id18} OR new_actor_id = {id19} OR new_actor_id = {id20}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE11,
    id1=pagerank_actor_top20[0],
    id2=pagerank_actor_top20[1],
    id3=pagerank_actor_top20[2],
    id4=pagerank_actor_top20[3],
    id5=pagerank_actor_top20[4],
    id6=pagerank_actor_top20[5],
    id7=pagerank_actor_top20[6],
    id8=pagerank_actor_top20[7],
    id9=pagerank_actor_top20[8],
    id10=pagerank_actor_top20[9],
    id11=pagerank_actor_top20[10],
    id12=pagerank_actor_top20[11],
    id13=pagerank_actor_top20[12],
    id14=pagerank_actor_top20[13],
    id15=pagerank_actor_top20[14],
    id16=pagerank_actor_top20[15],
    id17=pagerank_actor_top20[16],
    id18=pagerank_actor_top20[17],
    id19=pagerank_actor_top20[18],
    id20=pagerank_actor_top20[19]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [328]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE11))
data.head()

,new_actor_id,follower_rank
0,318510,3452
1,1172471,4982
2,2730410,19847
3,292688,61054
4,1389704,128125
5,1065416,128816
6,1103564,152899
7,4357576,154295
8,2263149,159060
9,4087589,170586


In [329]:
pagerank_actor_top20_infollow = data.to_pandas()
actor_pagerank_result_top20 = pd.DataFrame(actor_pagerank_result, columns=['new_actor_id','actor_name','rank'])[:20]
# 合并pagerank_actor_top20_infollow和actor_pagerank_result_top20
pagerank_follow_top20 = pd.merge(actor_pagerank_result_top20, pagerank_actor_top20_infollow, on='new_actor_id')
pagerank_follow_top20

,new_actor_id,actor_name,rank,follower_rank
0,1389704,direwolf-github,1,128125
1,1065416,xmo-odoo,2,128816
2,2730410,codacy-badger,4,19847
3,1172471,gitter-badger,5,4982
4,2263149,QualitySoftwareDeveloper,6,159060
5,2121899,idsb3t1,8,184369
6,5148049,codeserver-service-qa,9,173762
7,6575172,turbocanary-admin,10,180074
8,1103564,slskopytko,11,152899
9,6270054,shiftleft-staging,12,174154


##### top50

In [330]:
pagerank_actor_top50=actor_pagerank_info['new_actor_id'][:50].tolist()
print(pagerank_actor_top50)

[1389704, 1065416, 5631549, 2730410, 1172471, 2263149, 4823957, 2121899, 5148049, 6575172, 1103564, 6270054, 6282961, 292688, 4357576, 4087589, 58376, 5580835, 318510, 6806159, 4074427, 454546, 4439111, 6185714, 140539, 1178804, 989410, 5177482, 3693443, 640086, 477752, 492387, 1100073, 492993, 528756, 4487914, 3703474, 5710996, 15183, 3696396, 5074948, 2675763, 5829576, 618299, 4883463, 2654741, 2982042, 951365, 1799194, 2016173]


In [331]:
TEMP_EVAL_TABLE12 = 'hx_pagerank_follow_actor_top20'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE12
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220519123622772gr6ohv21>

In [332]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10} OR new_actor_id = {id11} OR new_actor_id = {id12}
        OR new_actor_id = {id13} OR new_actor_id = {id14} OR new_actor_id = {id15} OR new_actor_id = {id16}
        OR new_actor_id = {id17} OR new_actor_id = {id18} OR new_actor_id = {id19} OR new_actor_id = {id20}
        OR new_actor_id = {id21} OR new_actor_id = {id22} OR new_actor_id = {id23} OR new_actor_id = {id24}
        OR new_actor_id = {id25} OR new_actor_id = {id26} OR new_actor_id = {id27} OR new_actor_id = {id28}
        OR new_actor_id = {id29} OR new_actor_id = {id30} OR new_actor_id = {id31} OR new_actor_id = {id32}
        OR new_actor_id = {id33} OR new_actor_id = {id34} OR new_actor_id = {id35} OR new_actor_id = {id36}
        OR new_actor_id = {id37} OR new_actor_id = {id38} OR new_actor_id = {id39} OR new_actor_id = {id40}
        OR new_actor_id = {id41} OR new_actor_id = {id42} OR new_actor_id = {id43} OR new_actor_id = {id44}
        OR new_actor_id = {id45} OR new_actor_id = {id46} OR new_actor_id = {id47} OR new_actor_id = {id48}
        OR new_actor_id = {id49} OR new_actor_id = {id50}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE12,
    id1=pagerank_actor_top50[0], id2=pagerank_actor_top50[1], id3=pagerank_actor_top50[2], id4=pagerank_actor_top50[3],
    id5=pagerank_actor_top50[4], id6=pagerank_actor_top50[5], id7=pagerank_actor_top50[6], id8=pagerank_actor_top50[7],
    id9=pagerank_actor_top50[8], id10=pagerank_actor_top50[9], id11=pagerank_actor_top50[10], id12=pagerank_actor_top50[11],
    id13=pagerank_actor_top50[12], id14=pagerank_actor_top50[13], id15=pagerank_actor_top50[14], id16=pagerank_actor_top50[15],
    id17=pagerank_actor_top50[16], id18=pagerank_actor_top50[17], id19=pagerank_actor_top50[18], id20=pagerank_actor_top50[19],
    id21=pagerank_actor_top50[20], id22=pagerank_actor_top50[21], id23=pagerank_actor_top50[22], id24=pagerank_actor_top50[23],
    id25=pagerank_actor_top50[24], id26=pagerank_actor_top50[25], id27=pagerank_actor_top50[26], id28=pagerank_actor_top50[27],
    id29=pagerank_actor_top50[28], id30=pagerank_actor_top50[29], id31=pagerank_actor_top50[30], id32=pagerank_actor_top50[31],
    id33=pagerank_actor_top50[32], id34=pagerank_actor_top50[33], id35=pagerank_actor_top50[34], id36=pagerank_actor_top50[35],
    id37=pagerank_actor_top50[36], id38=pagerank_actor_top50[37], id39=pagerank_actor_top50[38], id40=pagerank_actor_top50[39],
    id41=pagerank_actor_top50[40], id42=pagerank_actor_top50[41], id43=pagerank_actor_top50[42], id44=pagerank_actor_top50[43],
    id45=pagerank_actor_top50[44], id46=pagerank_actor_top50[45], id47=pagerank_actor_top50[46], id48=pagerank_actor_top50[47],
    id49=pagerank_actor_top50[48], id50=pagerank_actor_top50[49]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [333]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE12))
data.head(10)

,new_actor_id,follower_rank
0,618299,564
1,454546,810
2,318510,3452
3,492387,4228
4,640086,4501
5,1172471,4982
6,1178804,5741
7,951365,12415
8,15183,12549
9,477752,17840


In [336]:
pagerank_actor_top50_infollow = data.to_pandas()
actor_pagerank_result_top50 = pd.DataFrame(actor_pagerank_result, columns=['new_actor_id','actor_name','rank'])[:50]
# 合并pagerank_actor_top50_infollow和actor_pagerank_result_top50
pagerank_follow_top50 = pd.merge(actor_pagerank_result_top50, pagerank_actor_top50_infollow, on='new_actor_id')
pagerank_follow_top50

,new_actor_id,actor_name,rank,follower_rank
0,1389704,direwolf-github,1,128125
1,1065416,xmo-odoo,2,128816
2,2730410,codacy-badger,4,19847
3,1172471,gitter-badger,5,4982
4,2263149,QualitySoftwareDeveloper,6,159060
5,2121899,idsb3t1,8,184369
6,5148049,codeserver-service-qa,9,173762
7,6575172,turbocanary-admin,10,180074
8,1103564,slskopytko,11,152899
9,6270054,shiftleft-staging,12,174154


#### correlation

##### repo

pagerank_watch_top10, pagerank_watch_top20, pagerank_watch_top50中，rank与watch_rank的spearman相关系数检验

In [338]:
print("pagerank表前10的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(pagerank_watch_top10['rank'].to_list(), pagerank_watch_top10['watch_rank'].to_list())[0])
print("pagerank表前20的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(pagerank_watch_top20['rank'].to_list(), pagerank_watch_top20['watch_rank'].to_list())[0])
print("pagerank表前50的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(pagerank_watch_top50['rank'].to_list(), pagerank_watch_top50['watch_rank'].to_list())[0])

pagerank表前10的repo的排名与其在watch中对应排名的correlation值： 0.19999999999999998
pagerank表前20的repo的排名与其在watch中对应排名的correlation值： 0.06766917293233082
pagerank表前50的repo的排名与其在watch中对应排名的correlation值： 0.09839135654261706


由于correlation在(0, 0.2)之间，故相关性弱。

birank得到的排名靠前的repo排名与watch统计信息中的排名几乎不相关，二者结果相差较大

##### actor

In [340]:
print("pagerank表前10的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(pagerank_follow_top10['rank'].to_list(), pagerank_follow_top10['follower_rank'].to_list())[0])
print("pagerank表前20的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(pagerank_follow_top20['rank'].to_list(), pagerank_follow_top20['follower_rank'].to_list())[0])
print("pagerank表前50的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(pagerank_follow_top50['rank'].to_list(), pagerank_follow_top50['follower_rank'].to_list())[0])

pagerank表前10的actor的排名与其在follow中对应排名的correlation值： 0.7142857142857144
pagerank表前20的actor的排名与其在follow中对应排名的correlation值： 0.34803921568627455
pagerank表前50的actor的排名与其在follow中对应排名的correlation值： -0.3669354838709677


与repo情况相似，birank得到的排名靠前的actor排名与follower统计信息中的排名几乎不相关

### top1000存csv后续做evaluation

In [346]:
# birank
birank_repo_top1000 = pd.DataFrame(repo_result, columns=['new_repo_id'])[:1000]
birank_repo_top1000.to_csv('./birank_repo_top1000.csv',index=False)
birank_actor_top1000 = pd.DataFrame(actor_result, columns=['new_actor_id'])[:1000]
birank_actor_top1000.to_csv('./birank_actor_top1000.csv',index=False)

In [347]:
#pagerank
pagerank_repo_top1000 = pd.DataFrame(repo_pagerank_result, columns=['new_repo_id'])[:1000]
pagerank_repo_top1000.to_csv('./pagerank_repo_top1000.csv',index=False)
pagerank_actor_top1000 = pd.DataFrame(actor_pagerank_result, columns=['new_actor_id'])[:1000]
pagerank_actor_top1000.to_csv('./pagerank_actor_top1000.csv',index=False)

In [348]:
#watch
birank_watch_top1000 = pd.DataFrame(pd_idwatch, columns=['new_repo_id'])[:1000]
birank_watch_top1000.to_csv('./birank_watch_top1000.csv',index=False)
#follow
birank_follow_top1000 = pd.DataFrame(pd_idfollow, columns=['new_actor_id'])[:1000]
birank_follow_top1000.to_csv('./birank_follow_top1000.csv',index=False)